In [ ]:
class Platform:
    WINDOWS = "windows"
    MACOS = "macos"

In [ ]:
root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling"
platform = Platform.WINDOWS

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import sys
sys.path.append(root_directory)

import os
from os.path import join, getsize
import threading
import time

import file_utils as fu

In [ ]:
json_dir_path = fu.relative_to_absolute_path(
    "GetAllSongHTML/song_list_link_by_artist/json",
    root_path=root_directory
)

print(json_dir_path)

In [ ]:
song_links_by_alphabet = fu.get_file_path_list_in_dir(json_dir_path)

song_links_by_alphabet = {
    path.split("/")[-1].split(".")[0].lower() : path
    for path in song_links_by_alphabet
}

print(song_links_by_alphabet)

In [ ]:
def song_link_to_relative_html_file_path(link):
    artist_letter = link.split("/")[-2][0]
    artist = link.split("/")[-2]
    song = link.split("/")[-1]

    return f"{artist_letter}/{artist}/{song}.html"


print(
    song_link_to_relative_html_file_path(
        fu.read_data_from_json_file(
            song_links_by_alphabet["a"])[0]["link"]
        )
    )

In [ ]:
def scroll_to_bottom(browser, scroll_pause_time = 0.5):
    # Get the screen height of the web
    screen_height = browser.execute_script("return window.screen.height;") 
    i = 1

    while True:
        # Scroll one screen height each time
        browser.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        time.sleep(scroll_pause_time)
        
        # Cập nhật scroll_height sau mỗi lần scroll, vì scroll_height có thể thay đổi sau khi scroll trang
        scroll_height = browser.execute_script("return document.body.scrollHeight;")

        i += 1
        if (screen_height) * i > scroll_height:
            break

In [ ]:
def scroll_and_get_html_content(url, webdriver_instance):
    webdriver_instance.get(url)

    scroll_to_bottom(webdriver_instance)
    
    contents = webdriver_instance.page_source

    return contents

In [ ]:
def get_song_raw_html(url, webdriver_instance):
    print(f"{url}: processing...")
    
    page_source = scroll_and_get_html_content(url, webdriver_instance)

    path_to_save_html = fu.relative_to_absolute_path(
        song_link_to_relative_html_file_path(url),
        root_path=fu.relative_to_absolute_path(
            "GetAllSongHTML/song_list/raw_html",
            root_path=root_directory
        )
    )
    
    fu.write_data_to_file(path_to_save_html, page_source)

    print(f"{url}: done")

    return None

In [ ]:
def open_edge_in_remote_debugging_mode(port, platform, browser_instance_data_dir):
    print(f"Opening Edge in remote debugging mode on port {port}...")

    macos_command = f'''open -na "Microsoft Edge.app" --args
                        --remote-debugging-port={port}
                        --user-data-dir="{browser_instance_data_dir}"'''

    windows_command = f'''start msedge.exe 
                        --remote-debugging-port={port} 
                        --user-data-dir="{browser_instance_data_dir}'''

    command = macos_command if platform == Platform.MACOS else windows_command
    
    os.system(command.replace("\n", " "))

In [ ]:
def init_selenium_in_remote_debugging_mode(port):
    options = webdriver.EdgeOptions()
    options.add_experimental_option("debuggerAddress", f"localhost:{port}")

    browser = webdriver.Edge(options=options)
    # browser.maximize_window()

    return browser

In [ ]:
class BrowserStatus:
    AVAILABLE = "available"
    BUSY = "busy"

In [ ]:
webdriver_pool = []

def init_webdriver_pool(n_webdriver=5):
    for i in range(n_webdriver):
        port = 40000 + i
        open_edge_in_remote_debugging_mode(
            port,
            platform,
            fu.relative_to_absolute_path(
                f"GetAllSongHTML/song_list/browser_instance_data/{port}",
                root_path=root_directory
            )
        )

        browser = init_selenium_in_remote_debugging_mode(port)

        webdriver_pool.append(
            {
                "port": port,
                "browser": browser,
                "browser_status": BrowserStatus.AVAILABLE
            }
        )

In [ ]:
def change_port_status(port, status):
    for browser in webdriver_pool:
        if browser["port"] == port:
            browser["browser_status"] = status
            break

In [ ]:
def get_raw_html_with_webdriver_from_pool(url, webdriver_pool_item):
    for browser in webdriver_pool:
        if (browser["port"] == webdriver_pool_item["port"] 
            and browser["browser_status"] == BrowserStatus.AVAILABLE):
            change_port_status(browser["port"], BrowserStatus.BUSY)
            get_song_raw_html(url, browser["browser"])
            change_port_status(browser["port"], BrowserStatus.AVAILABLE)
            break

In [ ]:
def get_song_raw_html_by_alphabet(letter, song_per_batch=5):
    # Lấy danh sách link của letter
    song_links = fu.read_data_from_json_file(song_links_by_alphabet[letter])

    while(len(song_links) > 0):
        current_song_per_batch = song_per_batch if song_per_batch <= len(song_links) else len(song_links)
        song_batch = [song_links.pop(0) for _ in range(current_song_per_batch)]

        threadBatch = []

        for webdriver_pool_item in webdriver_pool:
            if webdriver_pool_item["browser_status"] == BrowserStatus.AVAILABLE:
                threadBatch.insert(
                    0,
                    threading.Thread(
                        target=get_raw_html_with_webdriver_from_pool,
                        args=(song_batch.pop(0), webdriver_pool_item)
                    )
                )

                threadBatch[0].start()
        
        for i in range(len(threadBatch)):
            threadBatch[i].join()

In [ ]:
song_per_batch = 5

init_webdriver_pool(n_webdriver=song_per_batch)

In [ ]:
# get_song_raw_html_by_alphabet('a', song_per_batch)